In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
#if you do not have access to the twenty qubit machine, use the publically available backend instead
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute

from qiskit.providers.aer import UnitarySimulator
import qiskit.ignis.verification.tomography.basis.circuits

In [4]:
# Needed for functions
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

In [5]:
#We used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the publically available ibmqx2 backend instead
device = provider.get_backend('ibmqx2')
#if you have access to the twenty qubit machine, use this 20 qubit machine instead
#device = provider.get_backend('ibmq_boeblingen')
backend_monitor(device)
device

ibmqx2
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 8
    backend_version: 2.0.2
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    allow_object_storage: True
    n_registers: 1
    allow_q_circuit: False
    backend_name: ibmqx2
    quantum_volume: 8
    online_date: 2017-01-24T05:00:00+00:00
    memory: True
    credits_required: True
    allow_q_object: True
    open_pulse: False
    sample_name: sparrow
    url: None
    conditional: False
    max_shots: 8192
    description: 5 qubit device
    coupling_map: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [3, 4], [4, 2], [4, 3]]
    max_experiments: 75

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.28627 GHz / 34.92678 µs / 36.50602 µs / 0.0 / 0.00058 / 0.00117 / 0.014
    Q1 / 5.23782 GHz / 57.77644 

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

In [6]:
# Construct an empty quantum circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
#assertion circuit for arbitrary superposition state, because the measurement in state_tomography_circuit and the 
#measurement of ancilla qubit in our assertion function will cause conflict, we manually insert the assertion circuits
circ.h(qr[1])
circ.s(qr[1])
circ.z(qr[1])
circ.h(qr[0])
circ.u3(1.5708,3.1416,-3.1416, qr[0])
circ.u3(1.5708,4.7124,1.5708, qr[1])
circ.cx(qr[1],qr[0])
circ.u3(1.5708,6.2832,0, qr[0])
circ.u3(1.5708,1.5708,1.5708, qr[1])       
circ.h(qr[0])
tomocircuits=state_tomography_circuits(circ, qr, meas_labels='Pauli', meas_basis='Pauli')
trans_qc = transpile(tomocircuits, device, optimization_level = 3)

In [7]:
jobSim = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
psi_circ = jobSim.result().get_statevector(circ)
print(psi_circ)

[ 1.29872182e-06-3.89596988e-06j  7.07108080e-01+1.42854153e-05j
 -1.29858348e-06-3.89605098e-06j  1.68826968e-05-7.07105482e-01j]


In [8]:
shots = 8192
#execute the job on real device
job = execute(tomocircuits, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


In [9]:
tomo_state = StateTomographyFitter(job.result(), tomocircuits)

In [10]:
rho_bell = tomo_state.fit()
F_state = state_fidelity(psi_circ, rho_bell)
print('Fit Fidelity =', F_state)

Fit Fidelity = 0.9040536166731964
